In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

In [3]:
embedding=nn.Embedding(5,10)
embedding(torch.tensor([0,0,1,1]))



tensor([[-0.2830,  0.5444,  1.0640,  1.2131, -0.8034, -0.6937,  0.3677, -0.2235,
         -1.4157,  0.0482],
        [-0.2830,  0.5444,  1.0640,  1.2131, -0.8034, -0.6937,  0.3677, -0.2235,
         -1.4157,  0.0482],
        [-0.0429,  0.1000,  1.2207,  0.2911,  0.2989, -0.6273,  0.7560,  0.3700,
          1.0670, -1.8570],
        [-0.0429,  0.1000,  1.2207,  0.2911,  0.2989, -0.6273,  0.7560,  0.3700,
          1.0670, -1.8570]], grad_fn=<EmbeddingBackward0>)

In [4]:
num_class=10
embed_len=20
latent_dim=100
img_size=28*28
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.label_embedding=nn.Embedding(num_class,embed_len)
        self.model=nn.Sequential(
            nn.Linear(latent_dim+embed_len, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.Linear(512, img_size),
            nn.Tanh()
            
        )
    def forward(self, z, label):
        # print(label.shape)
        c=self.label_embedding(label)
        print(c.shape)
        print(z.shape)
        x=torch.cat([z,c],dim=1)
        print(x.shape)
        return self.model(x)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_Gen=Generator().to(device)
model_Gen



Generator(
  (label_embedding): Embedding(10, 20)
  (model): Sequential(
    (0): Linear(in_features=120, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=784, bias=True)
    (6): Tanh()
  )
)

In [5]:
batch=32
z=torch.randn(batch, latent_dim)
z.shape
z=torch.tensor(z)
label=torch.randint(1,num_class , (batch,))
label=torch.tensor(label)

model_Gen(z.to(device), label.to(device))


torch.Size([32, 20])
torch.Size([32, 100])
torch.Size([32, 120])


/tmp/ipykernel_13/4065182639.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z=torch.tensor(z)
/tmp/ipykernel_13/4065182639.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label=torch.tensor(label)


tensor([[ 0.0233,  0.1032,  0.0210,  ...,  0.0166, -0.0158,  0.0218],
        [ 0.0658,  0.0279, -0.0045,  ...,  0.0064,  0.0457, -0.0240],
        [ 0.0602,  0.0431, -0.0407,  ...,  0.1263, -0.0267, -0.1134],
        ...,
        [-0.0478,  0.0480,  0.0444,  ...,  0.1328,  0.0209, -0.0397],
        [ 0.1278,  0.0454, -0.0454,  ...,  0.0691,  0.0013,  0.0068],
        [ 0.1092,  0.0255, -0.0386,  ...,  0.1092, -0.0152, -0.0606]],
       grad_fn=<TanhBackward0>)

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.label_embd=nn.Embedding(num_class, embed_len)
        self.model=nn.Sequential(
            nn.Linear(img_size+embed_len, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024,1),
            nn.Sigmoid()
            
        )
    def forward(self, img, label):
        c=self.label_embd(label)
        x=torch.cat([img,c], dim=1)
        return self.model(x)


model_Dis=Discriminator().to(device)
model_Dis

Discriminator(
  (label_embd): Embedding(10, 20)
  (model): Sequential(
    (0): Linear(in_features=804, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [7]:
img=torch.rand(batch,28,28)
img.shape
# img=nn.Flatten(img)
type(img)

img.shape

img_gen=img.view(img.size(0),-1).to(device)
img_gen.shape

label=torch.randint(0,num_class,(batch,))

p=model_Dis(img_gen, label)
p.shape

torch.Size([32, 1])